Here are your complete and organized notes for the lesson **Manually Testing a Data Pipeline**, covering both **end-to-end validation** and **checkpoint testing**:

---

## ✅ Manually Testing a Data Pipeline — Notes

Testing ensures your data pipeline extracts, transforms, and loads data reliably. Manual validation techniques help maintain **data integrity**, **performance**, and **trust** in the pipeline.

---

### 1. 🧪 Why Test Data Pipelines?

- Ensures the pipeline behaves **as expected** before production.
- Helps catch:
  - Data **quality issues**
  - **Transformation errors**
  - **Loading failures**
- Saves future **debugging and maintenance** time.
- Many pipelines go untested—make yours an exception!

---

### 2. 🌐 Environments: Testing vs Production

| Testing Environment         | Production Environment          |
|----------------------------|----------------------------------|
| Uses **sample/mock data**  | Uses **live, production data**   |
| Safe for **experimentation** | Must be **stable and secure**    |
| Encourages **peer review** | Accessible by **data consumers** |

> 🔁 Always test your pipeline end-to-end in a testing environment **before** deploying to production.

---

### 3. 🔄 End-to-End Pipeline Testing

**Definition**: Simulates a full ETL run with test data.

🧩 Benefits:
- Checks **pipeline reliability** over multiple runs
- Enables **peer review** and feedback
- Helps test **user access** to output data

🛠️ Includes:
- Data is extracted, transformed, and loaded fully
- Sample data mimics structure of real production data
- Outputs are validated at every stage

---

### 4. 📍 Checkpoint Validation

**Checkpoint** = A point **between components** in the pipeline (e.g., after extract, after transform, or after load).

👀 Example:
- After loading `clean_stock_data` into SQL:
  
```python
loaded_data = pd.read_sql("SELECT * FROM clean_stock_data", con=engine)

# Check structure
print(loaded_data.shape)
print(loaded_data.head())
```

📌 Helps detect:
- Dropped or malformed records
- Unexpected column changes
- Broken data types

---

### 5. 🔁 DataFrame Validation

Validate **output vs source** using `.equals()`:

```python
clean_stock_data.equals(loaded_data)  # Returns True or False
```

🧠 This confirms:
- **Row counts** match
- **No data lost or changed**
- **Schema consistency** between original and loaded tables

---

### ✅ Summary Table

| Task                            | Method                          | Tool         |
|---------------------------------|----------------------------------|--------------|
| Load DataFrame from SQL         | `pd.read_sql()`                 | pandas       |
| Quick shape/spot-check          | `.shape`, `.head()`             | pandas       |
| Full equality check             | `.equals(other_df)`             | pandas       |
| End-to-end pipeline test        | Simulate full ETL               | Custom code  |
| Peer review + consumer testing  | Share test data results         | Collaboration |

---

### 🚀 Final Tip

Always **test in stages** (checkpoints), and also do a **full run** (end-to-end). This ensures both granular and holistic reliability before deploying a pipeline.

---

Let me know if you'd like a test script template for validating checkpoints or comparing DataFrames!

### **Validating a data pipeline at "checkpoints"**
In this exercise, you'll be working with a data pipeline that extracts tax data from a CSV file, creates a new column, filters out rows based on average taxable income, and persists the data to a parquet file.

pandas has been loaded as pd, and the extract(), transform(), and load() functions have already been defined. You'll use these functions to validate the data pipeline at various checkpoints throughout its execution.

In [ ]:
# Extract and transform tax_data
raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)

# Check the shape of the raw_tax_data DataFrame, compare to the clean_tax_data DataFrame
print(f"Shape of raw_tax_data: {raw_tax_data.shape}")
print(f"Shape of clean_tax_data: {clean_tax_data.shape}")


In [ ]:
raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)
load(clean_tax_data, "clean_tax_data.parquet")

print(f"Shape of raw_tax_data: {raw_tax_data.shape}")
print(f"Shape of clean_tax_data: {clean_tax_data.shape}")

# Read in the loaded data, observe the head of each
to_validate = pd.read_parquet("clean_tax_data.parquet")
print(clean_tax_data.head(3))
print(to_validate.head(3))


### **Validating a data pipeline at "checkpoints"**
In this exercise, you'll be working with a data pipeline that extracts tax data from a CSV file, creates a new column, filters out rows based on average taxable income, and persists the data to a parquet file.

pandas has been loaded as pd, and the extract(), transform(), and load() functions have already been defined. You'll use these functions to validate the data pipeline at various checkpoints throughout its execution.

In [ ]:

raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)
load(clean_tax_data, "clean_tax_data.parquet")

print(f"Shape of raw_tax_data: {raw_tax_data.shape}")
print(f"Shape of clean_tax_data: {clean_tax_data.shape}")

to_validate = pd.read_parquet("clean_tax_data.parquet")
print(clean_tax_data.head(3))
print(to_validate.head(3))

# Check that the DataFrames are equal
print(to_validate.equals(clean_tax_data))


### **Testing a data pipeline end-to-end**
In this exercise, you'll be working with the same data pipeline as before, which extracts, transforms, and loads tax data. You'll practice testing this pipeline end-to-end to ensure the solution can be run multiple times, without duplicating the transformed data in the parquet file.

pandas has been loaded as pd, and the extract(), transform(), and load() functions have already been defined.

In [ ]:
# Trigger the data pipeline to run three times
for attempt in range(0, 3):
	print(f"Attempt: {attempt}")
	raw_tax_data = extract("raw_tax_data.csv")
	clean_tax_data = transform(raw_tax_data)
	load(clean_tax_data, "clean_tax_data.parquet")
	
	# Print the shape of the cleaned_tax_data DataFrame
	print(f"Shape of clean_tax_data: {clean_tax_data.shape}")
    
# Read in the loaded data, check the shape
to_validate = pd.read_parquet("clean_tax_data.parquet")
print(f"Final shape of cleaned data: {to_validate.shape}")


Here are your detailed and structured notes for the lesson **Unit Testing a Data Pipeline**, including key concepts, code examples, and practical takeaways:

---

## ✅ Unit Testing a Data Pipeline — Notes

**Unit tests** are small, fast, and repeatable tests that ensure each part of your pipeline behaves correctly **in isolation**. They're a foundational tool for **robust, error-resistant pipelines**.

---

### 1. 🧪 What are Unit Tests?

- Code that **tests other code** for correctness.
- Common in **software engineering**, and equally powerful in **data engineering**.
- Should be written **before end-to-end** validation.
- Can test both:
  - **Functionality** of code
  - **Quality/structure** of data

---

### 2. 🔧 Unit Testing with `pytest`

- `pytest` is a Python library used to write and run unit tests.
- **Test functions** should start with `test_` to be discovered automatically by `pytest`.

🧪 Example:

```python
def test_transformed_data():
    assert isinstance(clean_stock_data, pd.DataFrame)
```

▶️ Run the test:
```bash
python -m pytest
```

If the assertion passes, the test is ✅ successful.

---

### 3. ✅ `assert` and `isinstance` Basics

```python
pipeline_type = "ETL"

# Check type
assert isinstance(pipeline_type, str)  # ✅

# Check value
assert pipeline_type == "ETL"  # ✅

# Failing example:
assert isinstance(pipeline_type, float)  # ❌ Raises AssertionError
```

- `assert`: Raises an error if the condition is **False**
- `isinstance(obj, type)`: Checks if `obj` is an instance of `type`

---

### 4. 🧪 Pytest Fixtures for Test Data

Fixtures help:
- **Reuse setup code**
- Provide **test data or objects**
- Keep tests **clean and readable**

Example:

```python
import pytest

@pytest.fixture
def clean_data():
    return pd.DataFrame({
        "open": [100.5, 110.3],
        "close": [105.2, 115.0],
        "volume": [1000, 1200],
        "symbol": ["AAPL", "MSFT"]
    })

def test_transformed_data(clean_data):
    assert isinstance(clean_data, pd.DataFrame)
```

---

### 5. 📊 Unit Testing DataFrames

You can **test the data itself**, not just the functions!

✅ Example tests:

```python
def test_column_count(clean_data):
    assert len(clean_data.columns) == 4

def test_open_column_positive(clean_data):
    assert (clean_data["open"] > 0).all()

def test_open_column_max(clean_data):
    assert clean_data["open"].max() < 200
```

🧠 Use built-in pandas methods like `.min()`, `.max()`, `.columns`, and `.shape`.

---

### ✅ Summary Table

| Concept                     | Tool/Method           | Example                              |
|----------------------------|------------------------|--------------------------------------|
| Unit test                  | `def test_...()`       | `def test_transformed_data()`        |
| Run all tests              | CLI                    | `python -m pytest`                   |
| Type check                 | `assert isinstance()`  | `assert isinstance(df, pd.DataFrame)`|
| Fixtures (reusable data)   | `@pytest.fixture`      | `def clean_data()`                   |
| Data validation            | Pandas assertions      | `assert (df["open"] > 0).all()`      |

---

### 🚀 Final Tip

Think of unit testing as building a **safety net**. It ensures small changes in your pipeline don't cause unexpected, costly errors downstream.

---

Let me know if you'd like a **unit test template** or want help writing test cases for your own pipeline!

### **Validating a data pipeline with assert**
To build unit tests for data pipelines, it's important to get familiar with the assert keyword, and the isinstance() function. In this exercise, you'll practice using these two tools to validate components of a data pipeline.

The functions extract() and transform() have been made available for you, along with pandas, which has been imported as pd. Both extract() and transform() return a DataFrame. Good luck!

In [ ]:
raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)

# Validate the number of columns in the DataFrame
assert len(clean_tax_data.columns) == 5


In [ ]:
raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)

# Determine if the clean_tax_data DataFrames take type pd.DataFrame
assert isinstance(clean_tax_data, pd.DataFrame)


In [ ]:
raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)

# Assert that clean_tax_data takes is an instance of a string
try:
 assert isinstance(clean_tax_data, str)
except Exception as e:
	print(e)



### **Writing unit tests with pytest**
In this exercise, you'll practice writing a unit test to validate a data pipeline. You'll use assert and other tools to build the tests, and determine if the data pipeline performs as it should.

The functions extract() and transform() have been made available for you, along with pandas, which has been imported as pd. You'll be testing the transform() function, which is shown below.
```python
def transform(raw_data):
    raw_data["average_taxable_income"] = raw_data["total_taxable_income"] / raw_data["number_of_firms"]
    clean_data = raw_data.loc[raw_data["average_taxable_income"] > 100, :]
    clean_data.set_index("industry_name", inplace=True)
    return clean_data
```

In [ ]:
import pytest

def test_transformed_data():
    raw_tax_data = extract("raw_tax_data.csv")
    clean_tax_data = transform(raw_tax_data)
    
    # Assert that the transform function returns a pd.DataFrame
    assert isinstance(clean_tax_data, pd.DataFrame)
    
    # Assert that the clean_tax_data DataFrame has more columns than the raw_tax_data DataFrame
    assert len(clean_tax_data.columns) > len(raw_tax_data.columns)


Here are your detailed, structured notes for the lesson **"Running a Data Pipeline in Production"**, covering architecture, execution, orchestration, and monitoring best practices.

---

## 🚀 Running a Data Pipeline in Production — Notes

This lesson ties everything together, showing how to **structure, execute, and scale** a data pipeline for production environments.

---

### 1. 🏗️ Data Pipeline Architecture Patterns

| Approach                        | Description |
|-------------------------------|-------------|
| **Single script file**         | Easy but cluttered. All ETL logic and execution code in one file. |
| **Modular architecture** ✅    | Separate **function definitions** and **execution logic** into separate files for clarity and reusability. |

**Better structure example:**

```bash
project/
│
├── pipeline_utils.py     # extract(), transform(), load()
├── run_pipeline.py       # imports and runs ETL using try/except
```

In `run_pipeline.py`:

```python
from pipeline_utils import extract, transform, load
import logging

logging.basicConfig(level=logging.INFO)

try:
    data = extract()
    clean_data = transform(data)
    load(clean_data)
    logging.info("Pipeline ran successfully.")
except Exception as e:
    logging.error(f"Pipeline failed: {e}")
```

---

### 2. 🔁 End-to-End Pipeline Execution

- Use **`try/except`** blocks for error handling
- Use **`logging`** to monitor pipeline status
- Run pipeline via CLI or scheduler in production

🔧 This setup:
- Is **simple**, yet powerful
- Gives a **basic monitoring & alerting** setup
- Works as the base for production deployment

---

### 3. 🧩 Orchestration Tools for Production

#### 💡 Why Use Orchestration Tools?

| Feature                     | Description |
|----------------------------|-------------|
| ⏰ **Scheduling**            | Run tasks at intervals (daily, hourly, etc.) |
| 🔁 **Retry on failure**     | Automatically re-run failed tasks |
| 🚨 **Alerting**             | Notify on errors (Slack, email, etc.) |
| ⚖️ **Resource scaling**     | Efficiently manage compute resources |
| 📈 **Monitoring UI**        | Visual dashboards for pipeline status |

---

### 4. 🛠️ Popular Orchestration Tools

| Tool        | Notes |
|-------------|-------|
| **Apache Airflow** 🏆 | Most popular (~40% usage). Open-source, rich UI, plugin support. |
| **Prefect**          | Pythonic, modern, easier to configure than Airflow. |
| **Dagster**          | Emphasizes data quality and observability. |

> 💡 Source: [State of Data Engineering – Seattle Data Guy](https://open.substack.com/pub/seattledataguy/p/the-state-of-data-engineering-part)

---

### 5. ⚙️ Airflow Example (Basic Idea)

```python
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

def extract(): ...
def transform(): ...
def load(): ...

dag = DAG('my_etl_pipeline', start_date=datetime(2024, 1, 1), schedule_interval='@daily')

task1 = PythonOperator(task_id='extract', python_callable=extract, dag=dag)
task2 = PythonOperator(task_id='transform', python_callable=transform, dag=dag)
task3 = PythonOperator(task_id='load', python_callable=load, dag=dag)

task1 >> task2 >> task3
```

---

### 🔚 Summary Table

| Key Concept                | Best Practice                            |
|---------------------------|-------------------------------------------|
| Code organization         | Separate logic (`pipeline_utils.py`) from execution (`run_pipeline.py`) |
| Logging and error handling| Use `try/except` and `logging`            |
| Orchestration             | Use Airflow, Prefect, or Dagster for scheduling, retries, alerting |
| Manual execution          | Via Python script (`python run_pipeline.py`) |
| Monitoring                | Set up alerts/logs or use orchestrator UI |

---

### ✨ Final Tip

When moving to production:
- **Automate everything**
- **Fail gracefully**
- **Log everything**
- **Monitor & alert proactively**

---

Let me know if you want a **sample folder structure** or a ready-made Airflow DAG template to test out!

### **Data pipeline architecture patterns**
When building data pipelines, it's best to separate the files where functions are being defined from where they are being run.

In this exercise, you'll practice importing components of a pipeline into memory before using these functions to run the pipeline end-to-end. The project takes the following format, where pipeline_utils stores the extract(), transform(), and load() functions that will be used run the pipeline.

```bash
 > ls
 etl_pipeline.py
 pipeline_utils.py
```

In [ ]:
# Import the extract, transform, and load functions from pipeline_utils
from pipeline_utils import extract, transform, load

# Run the pipeline end to end by extracting, transforming and loading the data
raw_tax_data = extract("raw_tax_data.csv")
clean_tax_data = transform(raw_tax_data)
load(clean_tax_data, "clean_tax_data.parquet")


### **Running a data pipeline end-to-end**
It's important to monitor the performance of a pipeline when running in production. Earlier in the course, you explored tools such as exception handling and logging. In this last exercise, we'll practice running a pipeline end-to-end, while monitoring for exceptions and logging performance.

In [ ]:
import logging
from pipeline_utils import extract, transform, load

logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

try:
	raw_tax_data = extract("raw_tax_data.csv")
	clean_tax_data = transform(raw_tax_data)
	load(clean_tax_data, "clean_tax_data.parquet")
    
	logging.info("Successfully extracted, transformed and loaded data.")  # Log a success message.
    
except Exception as e:
	logging.error(f"Pipeline failed with error: {e}")  # Log failure message
